<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PySpark-연습하기" data-toc-modified-id="PySpark-연습하기-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PySpark 연습하기</a></span><ul class="toc-item"><li><span><a href="#SparkSession-만들기" data-toc-modified-id="SparkSession-만들기-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>SparkSession 만들기</a></span></li><li><span><a href="#데이터-로드" data-toc-modified-id="데이터-로드-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>데이터 로드</a></span></li><li><span><a href="#간단한-EDA" data-toc-modified-id="간단한-EDA-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>간단한 EDA</a></span><ul class="toc-item"><li><span><a href="#결측치-체크" data-toc-modified-id="결측치-체크-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>결측치 체크</a></span></li><li><span><a href="#종속변수-클래스-분포-확인해보기" data-toc-modified-id="종속변수-클래스-분포-확인해보기-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>종속변수 클래스 분포 확인해보기</a></span></li><li><span><a href="#수치형-변수-기술통계량-살펴보기" data-toc-modified-id="수치형-변수-기술통계량-살펴보기-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>수치형 변수 기술통계량 살펴보기</a></span></li><li><span><a href="#class가-2인값은-양성이므로-0으로-4를-1로-값-바꾸기" data-toc-modified-id="class가-2인값은-양성이므로-0으로-4를-1로-값-바꾸기-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>class가 2인값은 양성이므로 0으로 4를 1로 값 바꾸기</a></span></li></ul></li><li><span><a href="#수치형-변수들-Scaling-하기-위해-우선-Feature들을-벡터로-전환!" data-toc-modified-id="수치형-변수들-Scaling-하기-위해-우선-Feature들을-벡터로-전환!-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>수치형 변수들 Scaling 하기 위해 우선 Feature들을 벡터로 전환!</a></span></li></ul></li></ul></div>

## PySpark 연습하기

- 위스콘신 cancer 데이터로 분류 문제 해결하기

In [9]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

### SparkSession 만들기

In [10]:
spark = SparkSession.builder\
        .appName('Cancer Classification using pyspark')\
        .getOrCreate()

### 데이터 로드

In [11]:
df = spark.read.csv('/Users/younghun/Desktop/gitrepo/data/pyspark/breast-cancer-wisconsin.data')
df.show(10)


+-------+---+---+---+---+---+---+---+---+---+----+
|    _c0|_c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|
+-------+---+---+---+---+---+---+---+---+---+----+
|1000025|  5|  1|  1|  1|  2|  1|  3|  1|  1|   2|
|1002945|  5|  4|  4|  5|  7| 10|  3|  2|  1|   2|
|1015425|  3|  1|  1|  1|  2|  2|  3|  1|  1|   2|
|1016277|  6|  8|  8|  1|  3|  4|  3|  7|  1|   2|
|1017023|  4|  1|  1|  3|  2|  1|  3|  1|  1|   2|
|1017122|  8| 10| 10|  8|  7| 10|  9|  7|  1|   4|
|1018099|  1|  1|  1|  1|  2| 10|  3|  1|  1|   2|
|1018561|  2|  1|  2|  1|  2|  1|  3|  1|  1|   2|
|1033078|  2|  1|  1|  1|  2|  1|  1|  1|  5|   2|
|1033078|  4|  2|  1|  1|  2|  1|  2|  1|  1|   2|
+-------+---+---+---+---+---+---+---+---+---+----+
only showing top 10 rows



In [13]:
# StructField, StrudType 메소드는 하단의 라이브러리에 속해있음
from pyspark.sql.types import *

# 스키마를 따로 지정해보기
strings = 'id clump_thickness cell_size cell_shape adhesion single_cell_size nuciel chromatin nucleoli mlitoses class'
fields = [StructField(string, FloatType(), True) for string in strings.split()]
schema = StructType(fields)

# 스키마 재정의해서 데이터 다시 로드해보기
df = spark.read.csv('/Users/younghun/Desktop/gitrepo/data/pyspark/breast-cancer-wisconsin.data',
                   schema=schema)
df.show(10)

+---------+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|       id|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|
+---------+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|1000025.0|            5.0|      1.0|       1.0|     1.0|             2.0|   1.0|      3.0|     1.0|     1.0|  2.0|
|1002945.0|            5.0|      4.0|       4.0|     5.0|             7.0|  10.0|      3.0|     2.0|     1.0|  2.0|
|1015425.0|            3.0|      1.0|       1.0|     1.0|             2.0|   2.0|      3.0|     1.0|     1.0|  2.0|
|1016277.0|            6.0|      8.0|       8.0|     1.0|             3.0|   4.0|      3.0|     7.0|     1.0|  2.0|
|1017023.0|            4.0|      1.0|       1.0|     3.0|             2.0|   1.0|      3.0|     1.0|     1.0|  2.0|
|1017122.0|            8.0|     10.0|      10.0|     8.0|             7.

### 간단한 EDA
#### 결측치 체크

In [16]:
# 로드한 데이터의 칼럼을 확인할 때는 Pandas와 유사하다!
print(df.columns, type(df.columns))

['id', 'clump_thickness', 'cell_size', 'cell_shape', 'adhesion', 'single_cell_size', 'nuciel', 'chromatin', 'nucleoli', 'mlitoses', 'class'] <class 'list'>


In [22]:
from pyspark.sql.functions import isnan, count

# isnan()으로 결측치 체크하는 방법
# when(조건, True일 때 값, False일 때 값)
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()


+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
| id|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|
+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|  0|              0|        0|         0|       0|               0|     0|        0|       0|       0|    0|
+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+



In [26]:
# isNull()로 결측치 체크하기
# df.select([count(when(df[c].isNull(), c)).alias(c) for c in df.columns]).show()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
| id|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|
+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|  0|              0|        0|         0|       0|               0|    16|        0|       0|       0|    0|
+---+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+



#### 종속변수 클래스 분포 확인해보기

- 2 = 양성
- 4 = 악성
- 클래스 불균형 상태

In [29]:
class_df = df.groupby('class').count()
class_df.show()

+-----+-----+
|class|count|
+-----+-----+
|  2.0|  458|
|  4.0|  241|
+-----+-----+



In [33]:
# 다른 방법으로 악성인 데이터 몇 개인지 확인해보기
df.select(sum(when(df['class'] == 4, 1)).alias('Number of malignant')).show()

+-------------------+
|Number of malignant|
+-------------------+
|                241|
+-------------------+



#### 수치형 변수 기술통계량 살펴보기
- 소수점이 너무 많아서 round 시키기

In [34]:
import pyspark.sql.functions as F

df.describe().select('summary',
                    F.round('id', 0).alias('id_r'),
                    F.round('clump_thickness', 2).alias('r_1'),
                    F.round('cell_size', 2).alias('r_2'),
                    F.round('cell_shape', 2).alias('r_3'),
                    F.round('adhesion', 2).alias('r_4'),
                    F.round('single_cell_size', 2).alias('r_5'),
                    F.round('nuciel', 2).alias('r_6'),
                    F.round('chromatin', 2).alias('r_7'),
                    F.round('nucleoli', 2).alias('r_8'),
                    F.round('mlitoses', 2).alias('r_9')).show(10)

+-------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|summary|       id_r|  r_1|  r_2|  r_3|  r_4|  r_5|  r_6|  r_7|  r_8|  r_9|
+-------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  count|      699.0|699.0|699.0|699.0|699.0|699.0|683.0|699.0|699.0|699.0|
|   mean|  1071704.0| 4.42| 3.13| 3.21| 2.81| 3.22| 3.54| 3.44| 2.87| 1.59|
| stddev|   617096.0| 2.82| 3.05| 2.97| 2.86| 2.21| 3.64| 2.44| 3.05| 1.72|
|    min|    61634.0|  1.0|  1.0|  1.0|  1.0|  1.0|  1.0|  1.0|  1.0|  1.0|
|    max|1.3454352E7| 10.0| 10.0| 10.0| 10.0| 10.0| 10.0| 10.0| 10.0| 10.0|
+-------+-----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+



In [48]:
# 클래스에 따라 가장 큰 cell_size, cell_shape 살펴보기
df.groupBy('class').agg({'cell_size':'avg', 'cell_shape':'avg'})\
  .select('class',
         F.round('avg(cell_size)', 2).alias('Average size of cell'),
         F.round('avg(cell_shape)', 2).alias('Average shape of cell')).show()

+-----+--------------------+---------------------+
|class|Average size of cell|Average shape of cell|
+-----+--------------------+---------------------+
|  2.0|                1.33|                 1.44|
|  4.0|                6.57|                 6.56|
+-----+--------------------+---------------------+



- groupBy+agg한 데이터프레임을 할당한 변수를 부르면 Nonetype으로 뜨기 때문에 한 번에 이어서 select 구문에서 Round를 시키자!

In [58]:
cols = df.columns
cols.pop()
cols.remove('id')
print(cols)

['clump_thickness', 'cell_size', 'cell_shape', 'adhesion', 'single_cell_size', 'nuciel', 'chromatin', 'nucleoli', 'mlitoses']


In [64]:
# 모든 수치형 변수들 클래스값에 따라 평균값 도출시켜보기
df.groupby('class').agg({col: 'avg' for col in cols}).show()

+-----+---------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|class|avg(single_cell_size)|     avg(adhesion)|avg(clump_thickness)|    avg(cell_size)|    avg(chromatin)|  avg(cell_shape)|     avg(nucleoli)|     avg(mlitoses)|       avg(nuciel)|
+-----+---------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  2.0|   2.1200873362445414|1.3646288209606987|  2.9563318777292578|1.3253275109170306|2.1004366812227073|1.443231441048035|1.2903930131004366|1.0633187772925765|1.3468468468468469|
|  4.0|   5.2987551867219915| 5.547717842323651|   7.195020746887967| 6.572614107883817| 5.979253112033195|6.560165975103734| 5.863070539419087|2.5892116182572615| 7.627615062761507|
+-----+---------------------+------------------+--------------------+----------------

#### class가 2인값은 양성이므로 0으로 4를 1로 값 바꾸기

In [68]:
df = df.withColumn('class',
             when(df['class'] == 2, 0).otherwise(1))
df.show()

+---------+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|       id|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|
+---------+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|1000025.0|            5.0|      1.0|       1.0|     1.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|
|1002945.0|            5.0|      4.0|       4.0|     5.0|             7.0|  10.0|      3.0|     2.0|     1.0|    0|
|1015425.0|            3.0|      1.0|       1.0|     1.0|             2.0|   2.0|      3.0|     1.0|     1.0|    0|
|1016277.0|            6.0|      8.0|       8.0|     1.0|             3.0|   4.0|      3.0|     7.0|     1.0|    0|
|1017023.0|            4.0|      1.0|       1.0|     3.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|
|1017122.0|            8.0|     10.0|      10.0|     8.0|             7.

In [69]:
# 필요없는 칼럼인 id drop하기
df = df.drop('id')
df.show(5)

+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|
+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
|            5.0|      1.0|       1.0|     1.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|
|            5.0|      4.0|       4.0|     5.0|             7.0|  10.0|      3.0|     2.0|     1.0|    0|
|            3.0|      1.0|       1.0|     1.0|             2.0|   2.0|      3.0|     1.0|     1.0|    0|
|            6.0|      8.0|       8.0|     1.0|             3.0|   4.0|      3.0|     7.0|     1.0|    0|
|            4.0|      1.0|       1.0|     3.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|
+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+
only showing top 5 rows



### 수치형 변수들 Scaling 하기 위해 우선 Feature들을 벡터로 전환!

- 피처들을 벡터로 전환할 때 결측치를 어떻게 처리하고 변환시킬지에 대한 ``setHandleInvalid`` 사용하기!
    * ``skip``, ``keep``
    * <a href='https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null'>관련링크 StackOverflow</a>

In [86]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cols,
                           outputCol='ftr_vector')
vectorized_df = assembler.setHandleInvalid('skip').transform(df)
vectorized_df.show(10, truncate=True)

+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+--------------------+
|clump_thickness|cell_size|cell_shape|adhesion|single_cell_size|nuciel|chromatin|nucleoli|mlitoses|class|          ftr_vector|
+---------------+---------+----------+--------+----------------+------+---------+--------+--------+-----+--------------------+
|            5.0|      1.0|       1.0|     1.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|[5.0,1.0,1.0,1.0,...|
|            5.0|      4.0|       4.0|     5.0|             7.0|  10.0|      3.0|     2.0|     1.0|    0|[5.0,4.0,4.0,5.0,...|
|            3.0|      1.0|       1.0|     1.0|             2.0|   2.0|      3.0|     1.0|     1.0|    0|[3.0,1.0,1.0,1.0,...|
|            6.0|      8.0|       8.0|     1.0|             3.0|   4.0|      3.0|     7.0|     1.0|    0|[6.0,8.0,8.0,1.0,...|
|            4.0|      1.0|       1.0|     3.0|             2.0|   1.0|      3.0|     1.0|     1.0|    0|[4.0,1

In [90]:
# 스케일링 적용하기 
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='ftr_vector',
                       outputCol='ftr_vector_scaled')
scaled_df = scaler.fit(vectorized_df).transform(vectorized_df)
scaled_df.select('ftr_vector', 'ftr_vector_scaled').show(5)

+--------------------+--------------------+
|          ftr_vector|   ftr_vector_scaled|
+--------------------+--------------------+
|[5.0,1.0,1.0,1.0,...|[1.77257110220912...|
|[5.0,4.0,4.0,5.0,...|[1.77257110220912...|
|[3.0,1.0,1.0,1.0,...|[1.06354266132547...|
|[6.0,8.0,8.0,1.0,...|[2.12708532265095...|
|[4.0,1.0,1.0,3.0,...|[1.41805688176730...|
+--------------------+--------------------+
only showing top 5 rows

